In [1]:
import pandas as pd
import requests
import folium

In [2]:
df_cities = pd.read_csv('cities_dataset.csv')
df_cities.head(10)

,ID,Name,Habitants,Index,Acreage,HabitantsPerSquareKm,ProsperityIndex,Province,Latitude,Longitude,BoundingBox
0,1,Antwerpen,525.935,443.000,"204,51",2.572,"88,2",Antwerpen,51.221110,4.399708,"['51.1432868', '51.3776412', '4.2175769', '4.4..."
1,2,Gent,262.219,204.000,"156,18",1.679,"102,3",Oost-Vlaanderen,51.053829,3.725012,"['50.9795422', '51.187946', '3.5797616', '3.84..."
2,3,Charleroi,202.267,370.000,"102,08",1.982,"73,0",Henegouwen,50.412033,4.443624,"['50.3527894', '50.4925149', '4.3474458', '4.5..."
3,4,Luik,197.327,219.000,"69,39",2.844,"81,4",Luik,50.645138,5.573420,"['50.5610182', '50.6881981', '5.5233883', '5.6..."
4,5,Brussel,181.726,140.000,"32,61",5.573,"70,0",Brussel,50.846557,4.351697,"['50.6865573', '51.0065573', '4.191697', '4.51..."
5,6,Schaarbeek,133.309,2.146,"8,14",16.377,"65,4",Brussel,50.867604,4.373712,"['50.8434069', '50.8811977', '4.3571322', '4.4..."
6,7,Anderlecht,119.714,2.007,"17,74",6.747,"63,8",Brussel,50.839098,4.329653,"['50.8070598', '50.850549', '4.2437658', '4.34..."
7,8,Brugge,118.325,194.000,"138,40",855.000,"111,7",West-Vlaanderen,51.208553,3.226772,"['51.1581918', '51.363347', '3.1341802', '3.30..."
8,9,Namen,110.779,260.000,"175,69",631.000,"99,9",Namen,50.466528,4.866189,"['50.3872825', '50.5313007', '4.723053', '4.98..."
9,10,Leuven,101.624,288.000,"56,63",1.794,"113,6",Vlaams-Brabant,50.879202,4.701168,"['50.8242096', '50.9440707', '4.640295', '4.77..."


In [3]:
df_credentials = pd.read_csv('credentials.txt',header=None)
CLIENT_ID = df_credentials[1][0]
CLIENT_SECRET = df_credentials[1][1]
VERSION = '20180605' # Foursquare API version

In [4]:
near = 'Lokeren, Belgium'
radius = 2000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            near, 
            radius, 
            LIMIT)

result = requests.get(url).json()
result["response"]['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4c5093d9bd099521bed1525e',
   'name': 'De Donkere Wolk',
   'location': {'address': 'Torenstraat 14',
    'lat': 51.105140425790516,
    'lng': 3.991317566213707,
    'labeledLatLngs': [{'label': 'display',
      'lat': 51.105140425790516,
      'lng': 3.991317566213707}],
    'postalCode': '9160',
    'cc': 'BE',
    'city': 'Lokeren',
    'state': 'Oost-Vlaanderen',
    'country': 'België',
    'formattedAddress': ['Torenstraat 14', '9160 Lokeren', 'België']},
   'categories': [{'id': '4bf58dd8d48988d116941735',
     'name': 'Bar',
     'pluralName': 'Bars',
     'shortName': 'Bar',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4c5093d9bd099521bed1525e-0'},
 {'reasons': {'cou

In [5]:
def getNearbyVenues(names, radius=2000):
    
    cities_found=[]
    cities_not_found = []
    cities_total = []
    venues_list=[]
    for name in names:
        #print(name)
        cities_total.append(name)
        near = '{}, Belgium'.format(name) 
        print(near)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            near, 
            radius, 
            LIMIT)
            
        # make the GET request
        #print(requests.get(url).json())
        found = False
        results = requests.get(url).json()
        if 'groups' in results["response"]:
            if(results["response"]['groups'] != None):
                if(results["response"]['groups'][0]['items'] != None):
                    item_list = results["response"]['groups'][0]['items']
                    if(item_list != None and len(item_list) > 0):
                        found = True
                    #print(results)
                    #print(' ')
                    # return only relevant information for each nearby venue
                    venues_list.append([(
                        name, 
                        v['venue']['name'], 
                        v['venue']['location']['lat'], 
                        v['venue']['location']['lng'],  
                        v['venue']['categories'][0]['name']) for v in item_list])
        
        if(found == True):
            cities_found.append(name)
        else:
            cities_not_found.append(name)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    print('Total cities: {} , {}'.format(len(cities_total),cities_total))
    print('Cities found: {} , {}'.format(len(cities_found),cities_found))
    print('Total cities: {} , {}'.format(len(cities_not_found),cities_not_found))

    return(nearby_venues)

In [6]:

be_venues = getNearbyVenues(names=df_cities['Name'])
be_venues

Antwerpen, Belgium
Gent, Belgium
Charleroi, Belgium
Luik, Belgium
Brussel, Belgium
Schaarbeek, Belgium
Anderlecht, Belgium
Brugge, Belgium
Namen, Belgium
Leuven, Belgium
Sint-Jans-Molenbeek, Belgium
Bergen, Belgium
Elsene, Belgium
Mechelen, Belgium
Aalst, Belgium
Ukkel, Belgium
La Louvière, Belgium
Hasselt, Belgium
Sint-Niklaas, Belgium
Kortrijk, Belgium
Oostende, Belgium
Doornik, Belgium
Genk, Belgium
Seraing, Belgium
Roeselare, Belgium
Moeskroen, Belgium
Sint-Lambrechts-Woluwe, Belgium
Vorst, Belgium
Verviers, Belgium
Jette, Belgium
Sint-Gillis, Belgium
Beveren, Belgium
Etterbeek, Belgium
Beringen, Belgium
Dendermonde, Belgium
Vilvoorde, Belgium
Turnhout, Belgium
Deinze*, Belgium
Dilbeek, Belgium
Heist-op-den-Berg, Belgium
Sint-Pieters-Woluwe, Belgium
Evere, Belgium
Lokeren, Belgium
Sint-Truiden, Belgium
Geel, Belgium
Eigenbrakel, Belgium
Herstal, Belgium
Halle, Belgium
Ninove, Belgium
Maasmechelen, Belgium
Waregem, Belgium
Brasschaat, Belgium
Grimbergen, Belgium
Mol, Belgium
Lier, B

Walhain, Belgium
Stavelot, Belgium
Beaumont, Belgium
Anhée, Belgium
Assesse, Belgium
Nieuwerkerken, Belgium
Chièvres, Belgium
Itter, Belgium
Hoegaarden, Belgium
Hensies, Belgium
Quiévrain, Belgium
Saint-Georges-sur-Meuse, Belgium
Sint-Laureins, Belgium
Pittem, Belgium
Herne, Belgium
Lierde, Belgium
Ramillies, Belgium
Villers-le-Bouillet, Belgium
Kluisbergen, Belgium
Moerbeke, Belgium
Kaprijke, Belgium
Wortegem-Petegem, Belgium
Braives, Belgium
Maarkedal, Belgium
Bekkevoort, Belgium
Engis, Belgium
Geetbets, Belgium
Hastière, Belgium
Elzele, Belgium
Limburg, Belgium
Remicourt, Belgium
Etalle, Belgium
Lobbes, Belgium
Lontzen, Belgium
Nandrin, Belgium
Lendelede, Belgium
Thimister-Clermont, Belgium
Pecq, Belgium
Drogenbos, Belgium
Celles, Belgium
Viroinval, Belgium
Somme-Leuze, Belgium
Vaux-sur-Sûre, Belgium
Bütgenbach, Belgium
Attert, Belgium
Florenville, Belgium
Saint-Hubert, Belgium
Hotton, Belgium
Nassogne, Belgium
Incourt, Belgium
Büllingen, Belgium
Amel, Belgium
Léglise, Belgium
March

,City,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Antwerpen,Moochie Frozen Yoghurt,51.220036,4.402850,Frozen Yogurt Shop
1,Antwerpen,Dogma Cocktails,51.221146,4.402854,Cocktail Bar
2,Antwerpen,Absinthbar,51.219912,4.400709,Cocktail Bar
3,Antwerpen,Hunkemöller,51.218611,4.405531,Lingerie Store
4,Antwerpen,Kartini Indonesisch Restaurant,51.219270,4.400557,Indonesian Restaurant
5,Antwerpen,Pitten en Bonen,51.217657,4.402712,Juice Bar
6,Antwerpen,Brasserie Appelmans,51.219879,4.400717,Cocktail Bar
7,Antwerpen,Quetzal,51.220625,4.402132,Coffee Shop
8,Antwerpen,Hendrik Consienceplein,51.221036,4.404180,Plaza
9,Antwerpen,Maison Tartine,51.221703,4.404996,Sandwich Place


In [7]:
be_venues

,City,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Antwerpen,Moochie Frozen Yoghurt,51.220036,4.402850,Frozen Yogurt Shop
1,Antwerpen,Dogma Cocktails,51.221146,4.402854,Cocktail Bar
2,Antwerpen,Absinthbar,51.219912,4.400709,Cocktail Bar
3,Antwerpen,Hunkemöller,51.218611,4.405531,Lingerie Store
4,Antwerpen,Kartini Indonesisch Restaurant,51.219270,4.400557,Indonesian Restaurant
5,Antwerpen,Pitten en Bonen,51.217657,4.402712,Juice Bar
6,Antwerpen,Brasserie Appelmans,51.219879,4.400717,Cocktail Bar
7,Antwerpen,Quetzal,51.220625,4.402132,Coffee Shop
8,Antwerpen,Hendrik Consienceplein,51.221036,4.404180,Plaza
9,Antwerpen,Maison Tartine,51.221703,4.404996,Sandwich Place


In [8]:
be_venues.to_csv('venues_dataset.csv',index=False)

In [9]:
print('There are {} venues in this dataset.'.format(be_venues.shape[0]))
be_venues.head()

There are 26771 venues in this dataset.


,City,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Antwerpen,Moochie Frozen Yoghurt,51.220036,4.402850,Frozen Yogurt Shop
1,Antwerpen,Dogma Cocktails,51.221146,4.402854,Cocktail Bar
2,Antwerpen,Absinthbar,51.219912,4.400709,Cocktail Bar
3,Antwerpen,Hunkemöller,51.218611,4.405531,Lingerie Store
4,Antwerpen,Kartini Indonesisch Restaurant,51.219270,4.400557,Indonesian Restaurant


In [10]:
res = be_venues.groupby('City')['Venue'].count()
#print(type(res))
#print(res.index)
print(res.shape)
for n,vc in zip(res.index,res):
    print(vc,'\t',n)

(570,)
28 	 's-Gravenbrakel
100 	 Aalst
92 	 Aalter*
50 	 Aarlen
100 	 Aarschot
93 	 Aartselaar
40 	 Aat
59 	 Affligem
82 	 Aiseau-Presles
38 	 Alken
26 	 Alveringem
10 	 Amay
5 	 Amel
27 	 Andenne
100 	 Anderlecht
11 	 Anderlues
12 	 Anhée
75 	 Ans
4 	 Anthisnes
11 	 Antoing
100 	 Antwerpen
43 	 Anzegem
44 	 Ardooie
43 	 Arendonk
46 	 As
94 	 Asse
41 	 Assenede
6 	 Assesse
4 	 Attert
4 	 Aubange
23 	 Aubel
55 	 Avelgem
51 	 Awans
15 	 Aywaille
95 	 Baarle-Hertog
47 	 Baelen
47 	 Balen
57 	 Bastenaken
12 	 Beaumont
19 	 Beauraing
66 	 Beernem
67 	 Beerse
44 	 Beersel
45 	 Begijnendijk
28 	 Bekkevoort
6 	 Belœil
99 	 Bergen
48 	 Beringen
47 	 Berlaar
52 	 Berlare
4 	 Berloz
5 	 Bernissart
30 	 Bertem
6 	 Bertogne
23 	 Bertrix
4 	 Bevekom
22 	 Bever
100 	 Beveren
25 	 Beyne-Heusay
34 	 Bierbeek
89 	 Bilzen
33 	 Binche
14 	 Bitsingen
7 	 Bièvre
100 	 Blankenberge
11 	 Blégny
32 	 Bocholt
60 	 Boechout
51 	 Bonheiden
81 	 Boom
64 	 Boortmeerbeek
38 	 Borgloon
47 	 Borgworm
81 	 Bornem
77 	

In [11]:
print('There are {} uniques categories.'.format(len(be_venues['Venue Category'].unique())))
be_venues['Venue Category'].unique()

There are 482 uniques categories.


array(['Frozen Yogurt Shop', 'Cocktail Bar', 'Lingerie Store',
       'Indonesian Restaurant', 'Juice Bar', 'Coffee Shop', 'Plaza',
       'Sandwich Place', 'Bar', 'Beer Bar', 'Fish & Chips Shop', 'Church',
       'Sushi Restaurant', 'Clothing Store', 'Kitchen Supply Store',
       'Cupcake Shop', 'Jazz Club', "Women's Store", 'Restaurant',
       'Asian Restaurant', 'Pub', 'Italian Restaurant',
       'Falafel Restaurant', 'Belgian Restaurant', 'Boutique',
       'Soup Place', 'Bistro', 'Ice Cream Shop', 'Spanish Restaurant',
       'Chinese Restaurant', 'Hotel', 'Shoe Store', 'Bookstore',
       'Chocolate Shop', 'Breakfast Spot', 'French Restaurant',
       'Tapas Restaurant', 'Candy Store', 'Taco Place', 'Donut Shop',
       'Optical Shop', 'Thai Restaurant', 'Shopping Plaza',
       'Concert Hall', 'Argentinian Restaurant', 'Design Studio',
       'Theater', 'Indie Movie Theater', 'Friterie',
       'Vegetarian / Vegan Restaurant', 'BBQ Joint',
       'Vietnamese Restaurant', 'Bak

In [12]:
# one hot encoding
be_onehot = pd.get_dummies(be_venues[['Venue Category']], prefix="", prefix_sep="")
#be_onehot.drop(columns=['City'],inplace=True)
print(be_onehot.shape)
be_onehot.head()

(26771, 482)


,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,...,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
be_onehot['City'] = be_venues['City'] 
fixed_columns = [be_onehot.columns[-1]] + list(be_onehot.columns[:-1])
be_onehot = be_onehot[fixed_columns]
be_onehot.head()

,City,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Antwerpen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Antwerpen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Antwerpen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Antwerpen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Antwerpen,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
be_grouped = be_onehot.groupby('City').mean()
# Neighborhood from index => column
be_grouped.reset_index(inplace=True)
be_grouped.head()

,City,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,Whisky Bar,Windmill,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,'s-Gravenbrakel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00,0.0,0.0,0.0
1,Aalst,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.02000,0.0,0.0,0.0,0.01,0.0,0.0,0.0
2,Aalter*,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.01087,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,Aarlen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,Aarschot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [15]:
num_top_venues = 5

for city in be_grouped['City']:
    print("----"+city+"----")
    temp = be_grouped[be_grouped['City'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----'s-Gravenbrakel----
         venue  freq
0  Supermarket  0.14
1     Platform  0.11
2     Friterie  0.11
3      Brewery  0.07
4      Stadium  0.04


----Aalst----
                venue  freq
0                 Bar  0.11
1         Coffee Shop  0.06
2      Clothing Store  0.05
3  Belgian Restaurant  0.04
4      Cosmetics Shop  0.03


----Aalter*----
                venue  freq
0                 Bar  0.10
1              Bakery  0.05
2         Supermarket  0.05
3  Belgian Restaurant  0.04
4            Friterie  0.04


----Aarlen----
                venue  freq
0                 Bar  0.12
1  Italian Restaurant  0.10
2         Supermarket  0.08
3         Pizza Place  0.04
4    Greek Restaurant  0.04


----Aarschot----
            venue  freq
0      Restaurant  0.07
1             Bar  0.06
2  Sandwich Place  0.06
3        Friterie  0.05
4          Bakery  0.04


----Aartselaar----
                    venue  freq
0          Clothing Store  0.08
1  Furniture / Home Store  0.06
2              

             venue  freq
0         Pharmacy  0.17
1       Restaurant  0.17
2   Scenic Lookout  0.17
3  Vacation Rental  0.17
4       Campground  0.17


----Bertrix----
           venue  freq
0            Bar  0.13
1     Restaurant  0.09
2    Supermarket  0.09
3  Train Station  0.04
4    Music Venue  0.04


----Bevekom----
         venue  freq
0  Supermarket  0.25
1          Bar  0.25
2   Sports Bar  0.25
3  Auto Garage  0.25
4       Palace  0.00


----Bever----
                venue  freq
0        Soccer Field  0.09
1     Bed & Breakfast  0.09
2                Farm  0.09
3         Supermarket  0.05
4  Athletics & Sports  0.05


----Beveren----
                venue  freq
0                 Bar  0.11
1         Supermarket  0.06
2            Friterie  0.05
3      Sandwich Place  0.05
4  Athletics & Sports  0.04


----Beyne-Heusay----
              venue  freq
0       Supermarket  0.12
1  Asian Restaurant  0.08
2       Pizza Place  0.08
3  Basketball Court  0.08
4            Bakery  0.08



                venue  freq
0              Bakery  0.18
1       Bowling Alley  0.09
2            Pharmacy  0.09
3                Pool  0.09
4  Italian Restaurant  0.09


----Dalhem----
            venue  freq
0     Pizza Place  0.17
1      Restaurant  0.17
2  Sandwich Place  0.17
3    Soccer Field  0.17
4          Bakery  0.17


----Damme----
                venue  freq
0          Restaurant  0.14
1  Belgian Restaurant  0.11
2              Bistro  0.06
3           Gastropub  0.06
4                 Bar  0.06


----Daverdisse----
             venue  freq
0            Hotel   0.2
1           Resort   0.2
2           Forest   0.2
3  Vacation Rental   0.2
4           Hostel   0.2


----De Haan----
        venue  freq
0         Bar  0.11
1       Hotel  0.09
2  Restaurant  0.07
3    Friterie  0.06
4      Bakery  0.04


----De Panne----
                          venue  freq
0  Theme Park Ride / Attraction  0.12
1                        Bakery  0.08
2                           Bar  0.07
3      

           venue  freq
0            Bar  0.25
1  Grocery Store  0.10
2    Supermarket  0.10
3     Campground  0.05
4   Gourmet Shop  0.05


----Flémalle----
                venue  freq
0         Supermarket  0.15
1         Pizza Place  0.10
2  Italian Restaurant  0.05
3      Sandwich Place  0.05
4          Restaurant  0.05


----Fléron----
                venue  freq
0         Supermarket  0.13
1          Restaurant  0.07
2  Italian Restaurant  0.07
3   French Restaurant  0.07
4                 Gym  0.03


----Fosses-la-Ville----
           venue  freq
0    Supermarket  0.21
1  Shopping Mall  0.07
2   Gourmet Shop  0.07
3   Soccer Field  0.07
4          Plaza  0.07


----Frameries----
                  venue  freq
0           Supermarket  0.20
1         Grocery Store  0.08
2         Train Station  0.08
3  Gym / Fitness Center  0.04
4             Pet Store  0.04


----Frasnes-lez-Anvaing----
                venue  freq
0                 Bar  0.08
1            Friterie  0.06
2  Belgian R

                    venue  freq
0                  Bakery  0.10
1  Furniture / Home Store  0.06
2             Supermarket  0.06
3           Grocery Store  0.06
4       Food & Drink Shop  0.06


----Herstappe----
               venue  freq
0  Recreation Center  0.12
1          Pet Store  0.12
2          Brasserie  0.12
3        Sports Club  0.12
4           Friterie  0.12


----Herve----
                venue  freq
0                 Bar  0.09
1         Supermarket  0.09
2  Italian Restaurant  0.09
3          Restaurant  0.06
4        Soccer Field  0.06


----Herzele----
         venue  freq
0          Bar  0.14
1  Supermarket  0.08
2     Friterie  0.06
3   Steakhouse  0.06
4       Bakery  0.06


----Heusden-Zolder----
              venue  freq
0          Friterie  0.08
1       Supermarket  0.06
2            Bakery  0.06
3  Kebab Restaurant  0.04
4    Cosmetics Shop  0.03


----Heuvelland----
                venue  freq
0          Restaurant  0.12
1                 Bar  0.11
2           

...

         venue  freq
0     Bus Stop  0.12
1       Bakery  0.12
2  Supermarket  0.07
3     Pharmacy  0.07
4     Friterie  0.07


----Luik----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.06
2   French Restaurant  0.05
3          Restaurant  0.05
4                 Bar  0.04


----Lummen----
          venue  freq
0   Supermarket  0.10
1      Friterie  0.07
2      Bus Stop  0.05
3           Bar  0.05
4  Soccer Field  0.05


----Léglise----
                  venue  freq
0        Soccer Stadium   0.2
1                Bakery   0.2
2  Other Great Outdoors   0.2
3                  Road   0.2
4              Friterie   0.2


----Maarkedal----
                     venue  freq
0                 Bus Stop  0.14
1          Bed & Breakfast  0.10
2                      Bar  0.10
3               Restaurant  0.07
4  Health & Beauty Service  0.03


----Maaseik----
                venue  freq
0          Restaurant  0.08
1         Supermarket  0.06
2  Athletics & Sports  0

            venue  freq
0             Bar  0.08
1        Friterie  0.08
2        Bus Stop  0.06
3  Sandwich Place  0.06
4     Supermarket  0.06


----Nieuwerkerken----
               venue  freq
0               Farm  0.05
1           Bus Stop  0.05
2  Convenience Store  0.05
3           Friterie  0.05
4             Bakery  0.05


----Nieuwpoort----
                venue  freq
0              Bistro  0.07
1  Belgian Restaurant  0.04
2          Restaurant  0.04
3                 Bar  0.04
4            Friterie  0.04


----Nijlen----
            venue  freq
0          Bakery  0.08
1  Sandwich Place  0.05
2     Supermarket  0.05
3        Friterie  0.05
4    Soccer Field  0.03


----Nijvel----
                 venue  freq
0    French Restaurant  0.06
1          Supermarket  0.06
2                  Bar  0.05
3  Sporting Goods Shop  0.05
4   Italian Restaurant  0.04


----Ninove----
                    venue  freq
0                Friterie  0.09
1                     Bar  0.08
2  Furniture / H

                venue  freq
0            Bus Stop  0.07
1  Athletics & Sports  0.07
2         Supermarket  0.07
3       Garden Center  0.07
4                 Spa  0.07


----Rijkevorsel----
                    venue  freq
0                     Bar  0.09
1      Athletics & Sports  0.09
2             Supermarket  0.06
3                     Pub  0.06
4  Furniture / Home Store  0.06


----Rixensart----
                venue  freq
0          Restaurant  0.09
1         Supermarket  0.07
2  Italian Restaurant  0.05
3              Bakery  0.05
4         Pizza Place  0.05


----Rochefort----
                venue  freq
0         Supermarket  0.13
1  Italian Restaurant  0.07
2               Hotel  0.07
3           Brasserie  0.07
4             Butcher  0.03


----Roeselare----
                venue  freq
0                 Bar  0.08
1  Belgian Restaurant  0.07
2                 Pub  0.05
3      Cosmetics Shop  0.04
4  Italian Restaurant  0.04


----Ronse----
                venue  freq
0         

             venue  freq
0           Bakery   0.2
1  Bed & Breakfast   0.2
2          Rafting   0.2
3      Golf Course   0.2
4            Hotel   0.2


----Tellin----
                   venue  freq
0                    Bar   0.4
1            Supermarket   0.2
2             Restaurant   0.2
3  Outdoors & Recreation   0.2
4                    ATM   0.0


----Temse----
                venue  freq
0                 Bar  0.06
1              Bakery  0.06
2         Supermarket  0.05
3            Friterie  0.04
4  Belgian Restaurant  0.04


----Tenneville----
                  venue  freq
0           Supermarket  0.17
1  Fast Food Restaurant  0.17
2               Stadium  0.17
3            Restaurant  0.17
4                Bakery  0.17


----Terhulpen----
               venue  freq
0  French Restaurant  0.08
1        Supermarket  0.05
2             Bakery  0.05
3   Asian Restaurant  0.03
4             Bistro  0.03


----Ternat----
          venue  freq
0           Bar  0.11
1   Supermarket  0.

          venue  freq
0  Tennis Court  0.12
1    Restaurant  0.12
2  Soccer Field  0.12
3      Friterie  0.12
4   Beer Garden  0.12


----Waterloo----
                venue  freq
0   French Restaurant  0.08
1  Italian Restaurant  0.08
2                 Gym  0.05
3              Bakery  0.04
4     Thai Restaurant  0.04


----Watermaal-Bosvoorde----
                venue  freq
0  Italian Restaurant  0.08
1   French Restaurant  0.07
2          Restaurant  0.06
3              Bakery  0.04
4      Ice Cream Shop  0.04


----Waver----
                venue  freq
0      Sandwich Place  0.07
1  Italian Restaurant  0.06
2              Bakery  0.04
3         Supermarket  0.04
4          Restaurant  0.03


----Weismes----
             venue  freq
0  Vacation Rental  0.21
1            Hotel  0.16
2           Bakery  0.11
3            Plaza  0.05
4   Tennis Stadium  0.05


----Welkenraedt----
                venue  freq
0         Supermarket  0.21
1  Chinese Restaurant  0.07
2                Road  0.

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['City'] = be_grouped['City']

for ind in np.arange(be_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(be_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(570, 11)


,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,'s-Gravenbrakel,Supermarket,Friterie,Platform,Brewery,Comedy Club,Stadium,Forest,Paintball Field,Gym / Fitness Center,Fast Food Restaurant
1,Aalst,Bar,Coffee Shop,Clothing Store,Belgian Restaurant,Cosmetics Shop,Bistro,Plaza,Friterie,Wine Bar,BBQ Joint
2,Aalter*,Bar,Bakery,Supermarket,Friterie,Belgian Restaurant,Tennis Court,Optical Shop,Brasserie,Pharmacy,Gym / Fitness Center
3,Aarlen,Bar,Italian Restaurant,Supermarket,Pizza Place,Friterie,Greek Restaurant,Eastern European Restaurant,Burger Joint,Japanese Restaurant,Lounge
4,Aarschot,Restaurant,Sandwich Place,Bar,Friterie,Supermarket,Bakery,Italian Restaurant,Pub,Coffee Shop,Park


In [18]:
from sklearn.cluster import KMeans

kclusters = 5

be_grouped_clustering = be_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(be_grouped_clustering)

print(len(kmeans.labels_))
print(kmeans.labels_)
# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 

570
[0 4 4 1 1 1 0 4 0 4 4 3 0 0 1 0 0 1 0 0 1 4 4 4 4 4 4 0 4 2 1 4 1 0 4 4 4
 1 0 0 4 4 4 4 4 1 1 4 4 4 4 0 4 3 4 0 4 4 1 4 4 4 1 3 1 1 4 1 4 4 4 4 1 4
 1 2 0 4 4 4 4 4 1 4 3 1 1 1 4 1 2 2 2 0 4 0 1 0 1 0 1 2 3 0 0 2 0 0 1 1 1
 1 1 1 2 4 1 4 4 1 4 4 4 4 4 4 4 4 4 4 1 1 0 4 1 0 1 4 2 1 4 4 1 1 1 1 4 0
 1 4 1 0 1 1 1 4 0 1 2 0 4 0 0 1 4 4 0 1 0 0 4 4 4 1 4 0 4 3 4 0 0 0 1 1 4
 1 1 4 4 4 4 0 0 1 4 0 4 4 3 4 4 4 3 4 0 3 4 1 4 1 2 0 4 4 4 4 3 2 4 4 4 4
 4 4 1 4 1 4 4 4 4 1 4 0 4 2 4 4 4 3 4 1 4 2 4 4 4 4 4 4 4 0 4 1 4 4 4 0 1
 1 4 4 4 1 4 0 1 1 0 0 4 1 4 1 1 0 1 4 4 4 4 1 1 4 4 0 1 2 4 4 4 4 4 1 0 4
 4 4 4 4 0 4 1 0 1 0 0 0 4 4 1 4 0 4 4 1 1 4 4 4 4 4 4 4 4 0 4 1 4 4 4 1 4
 1 4 4 1 1 0 1 4 4 1 4 4 3 4 4 3 4 4 4 4 3 0 4 1 2 4 4 4 1 0 4 1 4 0 4 0 1
 0 3 4 1 1 4 4 1 4 1 4 1 0 4 4 0 1 4 4 4 4 3 0 1 4 4 4 4 1 0 1 4 3 1 0 4 0
 0 1 4 0 4 1 4 0 1 0 1 3 1 4 4 4 2 2 4 4 4 1 1 1 4 4 4 4 4 0 4 0 1 1 4 1 1
 1 4 1 4 4 4 4 1 1 1 1 1 4 1 1 1 1 4 4 1 1 4 1 4 0 0 2 1 1 4 0 4 4 1 4 4 2
 4 4 0 1 4 4 4 3 4 4 

In [19]:
cols = list(df_cities.columns)
print(cols)
cols[1] = 'City'
print(cols)
df_cities.columns = cols
df_cities.head()


['ID', 'Name', 'Habitants', 'Index', 'Acreage', 'HabitantsPerSquareKm', 'ProsperityIndex', 'Province', 'Latitude', 'Longitude', 'BoundingBox']
['ID', 'City', 'Habitants', 'Index', 'Acreage', 'HabitantsPerSquareKm', 'ProsperityIndex', 'Province', 'Latitude', 'Longitude', 'BoundingBox']


,ID,City,Habitants,Index,Acreage,HabitantsPerSquareKm,ProsperityIndex,Province,Latitude,Longitude,BoundingBox
0,1,Antwerpen,525.935,443.0,"204,51",2.572,"88,2",Antwerpen,51.221110,4.399708,"['51.1432868', '51.3776412', '4.2175769', '4.4..."
1,2,Gent,262.219,204.0,"156,18",1.679,"102,3",Oost-Vlaanderen,51.053829,3.725012,"['50.9795422', '51.187946', '3.5797616', '3.84..."
2,3,Charleroi,202.267,370.0,"102,08",1.982,"73,0",Henegouwen,50.412033,4.443624,"['50.3527894', '50.4925149', '4.3474458', '4.5..."
3,4,Luik,197.327,219.0,"69,39",2.844,"81,4",Luik,50.645138,5.573420,"['50.5610182', '50.6881981', '5.5233883', '5.6..."
4,5,Brussel,181.726,140.0,"32,61",5.573,"70,0",Brussel,50.846557,4.351697,"['50.6865573', '51.0065573', '4.191697', '4.51..."


In [20]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
be_merged = df_cities.join(neighborhoods_venues_sorted.set_index('City'), on='City')

print(be_merged.shape)
be_merged.head()

(581, 22)


,ID,City,Habitants,Index,Acreage,HabitantsPerSquareKm,ProsperityIndex,Province,Latitude,Longitude,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Antwerpen,525.935,443.0,"204,51",2.572,"88,2",Antwerpen,51.221110,4.399708,...,Coffee Shop,Cocktail Bar,Clothing Store,Bar,French Restaurant,Bistro,Italian Restaurant,Belgian Restaurant,Plaza,Shoe Store
1,2,Gent,262.219,204.0,"156,18",1.679,"102,3",Oost-Vlaanderen,51.053829,3.725012,...,Coffee Shop,Bistro,Plaza,Bar,Breakfast Spot,Italian Restaurant,Vegetarian / Vegan Restaurant,Belgian Restaurant,Pub,Wine Bar
2,3,Charleroi,202.267,370.0,"102,08",1.982,"73,0",Henegouwen,50.412033,4.443624,...,Italian Restaurant,Clothing Store,Electronics Store,Restaurant,Brewery,Bar,Greek Restaurant,Fast Food Restaurant,Lounge,Coffee Shop
3,4,Luik,197.327,219.0,"69,39",2.844,"81,4",Luik,50.645138,5.573420,...,Italian Restaurant,Coffee Shop,Restaurant,French Restaurant,Bar,Gym / Fitness Center,Plaza,Bistro,Sandwich Place,Park
4,5,Brussel,181.726,140.0,"32,61",5.573,"70,0",Brussel,50.846557,4.351697,...,Bar,Bookstore,Chocolate Shop,Plaza,Beer Bar,Seafood Restaurant,Thai Restaurant,Coffee Shop,Belgian Restaurant,Bakery


In [21]:
print(df_cities.shape)
print(neighborhoods_venues_sorted.shape)
df1 = be_merged[be_merged.isna().any(axis=1)]
df1

(581, 11)
(570, 12)


,ID,City,Habitants,Index,Acreage,HabitantsPerSquareKm,ProsperityIndex,Province,Latitude,Longitude,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,38,Deinze*,43.571,155.0,"127,43",342.0,NaN,Oost-Vlaanderen,50.988662,3.506242,...,Bakery,Friterie,Italian Restaurant,Clothing Store,Gym,Sandwich Place,Optical Shop,Bistro,Shopping Mall,Trail
71,72,Pelt,32.902,967.0,"83,62",393.0,NaN,Limburg,51.219767,5.429950,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,102,Lievegem,26.162,133.0,"80,17",326.0,NaN,Oost-Vlaanderen,51.124810,3.585182,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,105,Puurs-Sint-Amands,25.882,210.0,"48,99",528.0,NaN,Antwerpen,51.066411,4.281235,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,123,Oudsbergen,23.532,781.0,"116,24",202.0,NaN,Limburg,51.093315,5.554075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,195,Fontaine-l'Evêque,17.806,330.0,"28,41",627.0,"84,4",Henegouwen,50.411779,4.324632,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,225,Kruisem,15.641,87.0,"70,69",221.0,NaN,Oost-Vlaanderen,50.908554,3.564892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,304,Veurne,11.903,107.0,"96,34",124.0,"99,8",West-Vlaanderen,51.071043,2.654886,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
327,328,Ecaussines,11.159,176.0,"34,77",321.0,"96,8",Henegouwen,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
347,348,Steenput,10.478,133.0,"31,75",330.0,"92,6",Henegouwen,50.705979,3.264018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
be_merged.dropna(inplace=True)
print(be_merged.shape)

(569, 22)


In [23]:
df1 = be_merged[be_merged.isna().any(axis=1)]
df1

,ID,City,Habitants,Index,Acreage,HabitantsPerSquareKm,ProsperityIndex,Province,Latitude,Longitude,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [24]:
import matplotlib.cm as cm
import matplotlib.colors as colors

latitude = 50.85045 - 0.40
longitude = 4.34878

# create map
map_clusters = folium.Map(location=[latitude + 0.05, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(be_merged['Latitude'], be_merged['Longitude'], be_merged['City'], be_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [32]:
df_cat_0 = be_merged.loc[be_merged['Cluster Labels'] == 0]
print(df_cat_0.shape)
df_cat_0

(90, 22)


,ID,City,Habitants,Index,Acreage,HabitantsPerSquareKm,ProsperityIndex,Province,Latitude,Longitude,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
55,56,Châtelet,35.903,487.0,"27,03",1.328,"74,7",Henegouwen,50.404608,4.524351,...,Supermarket,Grocery Store,Asian Restaurant,Pizza Place,Snack Place,Plaza,Food & Drink Shop,Food Court,Clothing Store,Gym / Fitness Center
83,84,Aat,29.311,114.0,"126,95",231.000,"100,9",Henegouwen,50.631135,3.776949,...,Supermarket,Italian Restaurant,Bar,Electronics Store,Steakhouse,Friterie,Event Space,Pet Store,Bakery,French Restaurant
92,93,Zinnik,28.016,161.0,"110,30",254.000,"95,6",Henegouwen,50.577531,4.073297,...,Supermarket,Bar,Italian Restaurant,Flower Shop,Chinese Restaurant,Brewery,Gym Pool,Sandwich Place,School,Fast Food Restaurant
94,95,Andenne,27.391,222.0,"86,17",318.000,"90,8",Namen,50.489398,5.096547,...,Supermarket,Bar,Friterie,Cosmetics Shop,Athletics & Sports,Soccer Field,Asian Restaurant,Platform,Electronics Store,Plaza
99,100,Flémalle,26.304,293.0,"36,68",717.000,"88,6",Luik,50.602281,5.458087,...,Supermarket,Pizza Place,Italian Restaurant,Restaurant,Sandwich Place,Ice Cream Shop,Discount Store,Clothing Store,French Restaurant,Sports Bar
103,104,Gembloers,26.014,222.0,"95,86",271.000,"110,3",Namen,50.559727,4.694313,...,Supermarket,French Restaurant,Friterie,Gastropub,Pizza Place,Furniture / Home Store,Shoe Store,Fast Food Restaurant,Bar,Clothing Store
128,129,Fleurus,22.918,227.0,"59,28",387.000,"87,4",Henegouwen,50.482944,4.549699,...,Supermarket,Fast Food Restaurant,Italian Restaurant,Road,Chinese Restaurant,Park,Train Station,Outdoors & Recreation,Greek Restaurant,Café
130,131,Grâce-Hollogne,22.660,231.0,"34,24",662.000,"83,7",Luik,50.644955,5.503152,...,Supermarket,Italian Restaurant,Sandwich Place,Pizza Place,Pet Store,Restaurant,Chinese Restaurant,Ice Cream Shop,Gas Station,Tennis Court
138,139,Frameries,21.934,172.0,"25,95",845.000,"84,6",Henegouwen,50.408757,3.890566,...,Supermarket,Grocery Store,Train Station,Furniture / Home Store,Pizza Place,Stadium,Steakhouse,Snack Place,Flower Shop,Belgian Restaurant
140,141,'s-Gravenbrakel,21.901,208.0,"84,68",259.000,"101,7",Henegouwen,50.612348,4.142514,...,Supermarket,Friterie,Platform,Brewery,Comedy Club,Stadium,Forest,Paintball Field,Gym / Fitness Center,Fast Food Restaurant


In [33]:
df_cat_1 = be_merged.loc[be_merged['Cluster Labels'] == 1]
print(df_cat_1.shape)
df_cat_1

(155, 22)


,ID,City,Habitants,Index,Acreage,HabitantsPerSquareKm,ProsperityIndex,Province,Latitude,Longitude,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Antwerpen,525.935,443.000,"204,51",2.572,"88,2",Antwerpen,51.221110,4.399708,...,Coffee Shop,Cocktail Bar,Clothing Store,Bar,French Restaurant,Bistro,Italian Restaurant,Belgian Restaurant,Plaza,Shoe Store
1,2,Gent,262.219,204.000,"156,18",1.679,"102,3",Oost-Vlaanderen,51.053829,3.725012,...,Coffee Shop,Bistro,Plaza,Bar,Breakfast Spot,Italian Restaurant,Vegetarian / Vegan Restaurant,Belgian Restaurant,Pub,Wine Bar
2,3,Charleroi,202.267,370.000,"102,08",1.982,"73,0",Henegouwen,50.412033,4.443624,...,Italian Restaurant,Clothing Store,Electronics Store,Restaurant,Brewery,Bar,Greek Restaurant,Fast Food Restaurant,Lounge,Coffee Shop
3,4,Luik,197.327,219.000,"69,39",2.844,"81,4",Luik,50.645138,5.573420,...,Italian Restaurant,Coffee Shop,Restaurant,French Restaurant,Bar,Gym / Fitness Center,Plaza,Bistro,Sandwich Place,Park
4,5,Brussel,181.726,140.000,"32,61",5.573,"70,0",Brussel,50.846557,4.351697,...,Bar,Bookstore,Chocolate Shop,Plaza,Beer Bar,Seafood Restaurant,Thai Restaurant,Coffee Shop,Belgian Restaurant,Bakery
5,6,Schaarbeek,133.309,2.146,"8,14",16.377,"65,4",Brussel,50.867604,4.373712,...,Turkish Restaurant,Snack Place,Bar,Pizza Place,Kebab Restaurant,Middle Eastern Restaurant,Supermarket,Gastropub,Electronics Store,Italian Restaurant
6,7,Anderlecht,119.714,2.007,"17,74",6.747,"63,8",Brussel,50.839098,4.329653,...,Bar,Supermarket,Sandwich Place,Sports Bar,Greek Restaurant,Snack Place,French Restaurant,Plaza,Grocery Store,Italian Restaurant
7,8,Brugge,118.325,194.000,"138,40",855.000,"111,7",West-Vlaanderen,51.208553,3.226772,...,Beer Bar,Hotel,Wine Bar,Bar,French Restaurant,Chocolate Shop,Clothing Store,Italian Restaurant,Tapas Restaurant,Cocktail Bar
8,9,Namen,110.779,260.000,"175,69",631.000,"99,9",Namen,50.466528,4.866189,...,Bar,Hotel,French Restaurant,Plaza,Fast Food Restaurant,Bookstore,Bakery,Thai Restaurant,Italian Restaurant,Gym / Fitness Center
9,10,Leuven,101.624,288.000,"56,63",1.794,"113,6",Vlaams-Brabant,50.879202,4.701168,...,Bar,Coffee Shop,Italian Restaurant,Plaza,Sushi Restaurant,Cosmetics Shop,Cocktail Bar,Ice Cream Shop,Mexican Restaurant,Bakery


In [34]:
df_cat_2 = be_merged.loc[be_merged['Cluster Labels'] == 2]
print(df_cat_2.shape)
df_cat_2

(24, 22)


,ID,City,Habitants,Index,Acreage,HabitantsPerSquareKm,ProsperityIndex,Province,Latitude,Longitude,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
201,202,Aubange,17.093,433.0,"45,60",375.0,"88,6",Luxemburg,49.567195,5.804499,...,Train Station,Border Crossing,Casino,Hotel,Zoo Exhibit,Food & Drink Shop,Festival,Field,Fish & Chips Shop,Fish Market
318,319,Durbuy,11.395,144.0,"156,61",73.0,"90,4",Luxemburg,50.352412,5.456248,...,Hotel,Belgian Restaurant,Restaurant,Recreation Center,Brasserie,Cocktail Bar,Bed & Breakfast,Campground,Rock Climbing Spot,Pool
431,432,Weismes,7.425,147.0,"96,93",77.0,"90,4",Luik,50.414913,6.111754,...,Vacation Rental,Hotel,Bakery,Brasserie,Restaurant,Supermarket,Soccer Field,Snack Place,Tennis Stadium,Bar
468,469,Hastière,6.022,227.0,"56,46",107.0,"82,9",Namen,50.196987,4.822430,...,Resort,Bed & Breakfast,Bakery,Belgian Restaurant,Grocery Store,Café,Cave,Vacation Rental,Canal Lock,Pub
471,472,Remicourt,5.906,220.0,"22,58",262.0,"112,3",Luik,50.680715,5.326717,...,Snack Place,Food Truck,Pharmacy,Train Station,Bakery,Hotel,Café,Ice Cream Shop,Basketball Court,Supermarket
482,483,Somme-Leuze,5.641,152.0,"95,09",59.0,"93,0",Namen,50.336585,5.366902,...,Hotel,Boarding House,Grocery Store,Travel Agency,French Restaurant,Bed & Breakfast,Tennis Court,Golf Course,Food & Drink Shop,Food
484,485,Bütgenbach,5.610,141.0,"97,31",58.0,"91,8",Luik,50.426726,6.205357,...,Hotel,Belgian Restaurant,Lake,Athletics & Sports,Supermarket,Coffee Shop,Bakery,Pool,Cocktail Bar,Bar
491,492,Büllingen,5.478,121.0,"150,49",36.0,"89,8",Luik,50.407553,6.257827,...,Hotel,Friterie,Supermarket,Bakery,Italian Restaurant,Zoo Exhibit,Fishing Store,Fondue Restaurant,Flower Shop,Flea Market
495,496,Bouillon,5.411,79.0,"149,09",36.0,"89,0",Luxemburg,49.794984,5.067253,...,Hotel,Belgian Restaurant,Italian Restaurant,Food,French Restaurant,Bistro,History Museum,Bar,Lounge,Bakery
505,506,Chiny,5.180,86.0,"113,69",46.0,"95,1",Luxemburg,49.738581,5.339425,...,Hotel,River,Belgian Restaurant,Campground,Athletics & Sports,Steakhouse,Rafting,Fish & Chips Shop,Fish Market,Fishing Spot


In [35]:
df_cat_3 = be_merged.loc[be_merged['Cluster Labels'] == 3]
print(df_cat_3.shape)
df_cat_3

(22, 22)


,ID,City,Habitants,Index,Acreage,HabitantsPerSquareKm,ProsperityIndex,Province,Latitude,Longitude,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
246,247,Amay,14.412,262.0,"27,61",522.0,"92,7",Luik,50.549776,5.324099,...,Supermarket,Train Station,Basketball Stadium,Basketball Court,Convenience Store,Restaurant,Field,Fish & Chips Shop,Fish Market,Food & Drink Shop
264,265,Ham-sur-Heure-Nalinnes,13.623,240.0,"45,68",298.0,"119,5",Henegouwen,50.317162,4.380322,...,Restaurant,Plaza,Bakery,Arts & Entertainment,Pizza Place,Cocktail Bar,Historic Site,Train Station,Fish & Chips Shop,Fish Market
299,300,Theux,12.031,141.0,"83,36",144.0,"107,0",Luik,50.525861,5.819896,...,Restaurant,Train Station,Stadium,Cave,Italian Restaurant,Castle,Sports Bar,Trail,Supermarket,Campground
319,320,Virton,11.332,160.0,"94,49",120.0,"94,7",Luxemburg,49.567726,5.532956,...,Supermarket,Soccer Stadium,Restaurant,Stadium,Tennis Court,Clothing Store,Tennis Stadium,Campground,French Restaurant,Train Station
339,340,Raeren,10.759,207.0,"74,21",145.0,"109,0",Luik,50.676130,6.110840,...,Restaurant,Train Station,Bowling Green,Museum,Bus Stop,Tennis Court,Flower Shop,Food & Drink Shop,Food,Fondue Restaurant
405,406,Habay,8.409,114.0,"103,64",81.0,"107,6",Luxemburg,49.735580,5.593251,...,Restaurant,Supermarket,Snack Place,Bookstore,Indie Movie Theater,Candy Store,Grocery Store,Pharmacy,Fishing Spot,Fish Market
408,409,Opzullik,8.324,94.0,"67,68",123.0,"110,1",Henegouwen,50.648454,3.923592,...,Restaurant,Brewery,Supermarket,Market,Rock Club,Food,Train Station,Sports Club,Field,Fish & Chips Shop
410,411,Messancy,8.218,200.0,"52,43",157.0,"116,7",Luxemburg,49.594986,5.814574,...,Restaurant,Train Station,Athletics & Sports,Buffet,Fishing Store,Food,Fondue Restaurant,Flower Shop,Flea Market,Zoo Exhibit
435,436,Hamois,7.357,190.0,"76,42",96.0,"100,0",Namen,50.340869,5.159447,...,Restaurant,Pharmacy,Friterie,Soccer Field,Pizza Place,Tennis Court,Fishing Spot,Flower Shop,Flea Market,Fishing Store
447,448,Chièvres,6.916,103.0,"46,91",147.0,"96,4",Henegouwen,50.587982,3.805689,...,Food & Drink Shop,Supermarket,Restaurant,Plaza,Zoo Exhibit,Fast Food Restaurant,Festival,Field,Fish & Chips Shop,Fish Market


In [36]:
df_cat_4 = be_merged.loc[be_merged['Cluster Labels'] == 4]
print(df_cat_4.shape)
df_cat_4

(278, 22)


,ID,City,Habitants,Index,Acreage,HabitantsPerSquareKm,ProsperityIndex,Province,Latitude,Longitude,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,15,Aalst,86.445,263.000,"78,12",1.107,"107,6",Oost-Vlaanderen,50.938323,4.039215,...,Bar,Coffee Shop,Clothing Store,Belgian Restaurant,Cosmetics Shop,Bistro,Plaza,Friterie,Wine Bar,BBQ Joint
25,26,Moeskroen,58.474,426.000,"40,08",1.459,"84,3",Henegouwen,50.743335,3.213909,...,Bar,Supermarket,French Restaurant,Bakery,Restaurant,Belgian Restaurant,Sandwich Place,Soccer Field,Cosmetics Shop,Lounge
31,32,Beveren,48.668,197.000,"150,18",324.000,"113,7",Oost-Vlaanderen,51.212885,4.249081,...,Bar,Supermarket,Sandwich Place,Friterie,Bakery,Athletics & Sports,Stables,Restaurant,Gastropub,Bus Stop
33,34,Beringen,46.346,588.000,"78,30",592.000,"94,4",Limburg,51.050203,5.220809,...,Supermarket,Bakery,Clothing Store,Italian Restaurant,Platform,Bookstore,Bus Stop,Athletics & Sports,Shoe Store,Bar
34,35,Dendermonde,45.769,227.000,"55,67",822.000,"108,3",Oost-Vlaanderen,51.031229,4.098112,...,Friterie,Bar,Shoe Store,Clothing Store,Brasserie,Supermarket,Restaurant,Pub,Sandwich Place,Electronics Store
35,36,Vilvoorde,44.727,651.000,"21,48",2.083,"96,0",Vlaams-Brabant,50.928137,4.432905,...,Bar,Friterie,Cosmetics Shop,Restaurant,Supermarket,Clothing Store,Snack Place,Chinese Restaurant,Sandwich Place,Brasserie
38,39,Dilbeek,42.847,637.000,"41,18",1.041,"119,8",Vlaams-Brabant,50.844155,4.265675,...,Bar,Friterie,Pizza Place,Bakery,Supermarket,Restaurant,Health & Beauty Service,Italian Restaurant,Brewery,Sandwich Place
39,40,Heist-op-den-Berg,42.681,352.000,"86,46",494.000,"108,0",Antwerpen,51.062516,4.719431,...,Bar,Pub,Friterie,Cosmetics Shop,Restaurant,Italian Restaurant,Electronics Store,Shoe Store,Chinese Restaurant,Bakery
42,43,Lokeren,41.690,194.000,"67,50",618.000,"102,7",Oost-Vlaanderen,51.104216,3.991111,...,Bakery,Bar,Friterie,Doner Restaurant,Soccer Field,Brasserie,Belgian Restaurant,Gym / Fitness Center,Soccer Stadium,Park
43,44,Sint-Truiden,40.590,227.000,"106,90",380.000,"103,8",Limburg,50.826948,5.203425,...,Bar,Friterie,Brasserie,Pizza Place,Supermarket,Ice Cream Shop,Wine Bar,Italian Restaurant,Bakery,Clothing Store
